In [13]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy.linalg as LA
from sklearn import linear_model
import matplotlib.pyplot as plt
import random
import math
# import seaborn as sns
%matplotlib inline

In [3]:
# Load the dataset
df = pd.read_csv('./merged_2011_PP.csv', encoding='latin1', sep=',')

In [4]:
#### Handle NaN

# Numerical : Replace with median
for i in range(len(df.columns)):
    typ= df[df.columns[i]].dtype
    if( typ =='float' or typ =='int' or typ=='bool' ):
        df[df.columns[i]].fillna(df[df.columns[i]].median(),inplace = True)

# Categorical : With most frequent
for i in range(len(df.columns)):
    typ= df[df.columns[i]].dtype
    if( typ =='object'):
        df[df.columns[i]].fillna(df[df.columns[i]].value_counts().idxmax(),inplace = True)

df=df.fillna(0) ## Filled all NaN with mean
#### Handle Privacy Suppressed        
df=df.replace(to_replace='PrivacySuppressed',value=0)

In [5]:
### Delete Non Informative Attributes (String type)
del df['INSTNM']
del df['CITY']
del df['STABBR']
del df['AccredAgency']
del df['INSTURL']
del df['NPCURL']
del df['REPAY_DT_MDN'] ## del Date
del df['SEPAR_DT_MDN'] ## del Date
del df['ZIP']          ## Some others
del df['UNITID']

# df1=df[['DEP_INC_PCT_LO','PREDDEG','PCIP12','NPT41_PUB','NPT42_PUB','UGDS_ASIAN','FIRSTGEN_WDRAW_ORIG_YR3_RT','md_earn_wne_p6']]
for i in range(len(df.columns)): ## Convert datatypes as integers from string
    df[df.columns[i]]= pd.to_numeric(df[df.columns[i]])

y= df['md_earn_wne_p6']
del df['md_earn_wne_p6']

y2=df['GRAD_DEBT_MDN']
del df['GRAD_DEBT_MDN']

In [6]:
########Remove redundant features (manual selection) 

with open('to_remove.txt','r') as f: ## to_remove.txt contains list of features to remove
    reader = f.read().splitlines()
    bad_features=reader ## list of features that are to be removed 
    
# bad_features.index('COMPL_RPY_1YR_RT')   
for i in range(len(bad_features)):  #now removing them
#     print("Removing",bad_features[i],"For i=",i)
    del df[bad_features[i]]         #del all bad features


In [7]:
## TRIM USELESS FEATURES 

## Find out the features in which same value occurs many times
copydf=df.copy(deep=True)
# counter=0
for i in range(len(copydf.columns)):
    current=copydf[copydf.columns[i]]
    typ=current.dtype
    if(typ=='int64' or typ=='float64'):
        count=current.value_counts()
        count=list(count)
        if(len(count)>0 and count[0]>7000):
#                 counter+=1
#             print("Attribute=",df.columns[i],"Freq=",count[0])
                del df[copydf.columns[i]]  #!!!!!!! should be df[copydf.columns[i]]

print(df.columns) ## 458 left now


Index(['PREDDEG', 'HIGHDEG', 'CONTROL', 'ADM_RATE', 'SATVR25', 'SATVR75',
       'SATMT25', 'SATMT75', 'SATWR25', 'SATWR75',
       ...
       'NOT1STGEN_WDRAW_2YR_TRANS_YR8_RT', 'NOT1STGEN_ENRL_4YR_TRANS_YR8_RT',
       'NOT1STGEN_ENRL_2YR_TRANS_YR8_RT', 'NOT1STGEN_UNKN_ORIG_YR8_RT',
       'NOT1STGEN_UNKN_4YR_TRANS_YR8_RT', 'NOT1STGEN_UNKN_2YR_TRANS_YR8_RT',
       'DEP_INC_PCT_LO', 'PAR_ED_PCT_MS', 'PAR_ED_PCT_HS', 'PAR_ED_PCT_PS'],
      dtype='object', length=458)


In [8]:
x=df.copy(deep=True)

## Train Data
x_train=x[:6000]
y_train=y[:6000]
y_train=y_train.reshape(-1,1)
y2_train=y2[:6000]
y2_train=y2_train.reshape(-1,1)

## Test Data
x_test=x[6000:]
y_test=y[6000:]
y_test=y_test.reshape(-1,1)
y2_test=y2[6000:]
y2_test=y2_test.reshape(-1,1)

In [9]:
######## Dimensionality Reduction Techniques

## Simple PCA
# from sklearn.decomposition import PCA
# pca = PCA(n_components=100)
# pca.fit(df)
# df=pca.transform(df)
# df.shape

# # Kernel PCA
# from sklearn.decomposition import PCA, KernelPCA
# kpca = KernelPCA(kernel="rbf",gamma=10,n_components=400)
# fit=kpca.fit(x_train)
# x_train = kpca.transform(x_train)
# x_test = kpca.transform(x_test)

### Plot
# plt.scatter(Z[:,0], Z[:,1])
# plt.show()


In [102]:
######## Linear Regression (Scikit--To check)
############################################
regr = linear_model.LinearRegression()

regr.fit(x_train,y_train) ## Train
result=regr.predict(x_test)  ## Test

## Mean Absolute Percentage Error
from numpy import inf
def mean_absolute_percentage_error(y_true, y_pred):
    a=np.abs(y_true - y_pred)
    a[a == inf] = 0
    b=( a/ y_true)
    b[b == inf] = 0
    b=np.mean(b)
    return b * 100

percent_error=mean_absolute_percentage_error(y_test,result)
print("Mean Absolute % error = ",percent_error)


## RMSE
from sklearn.metrics import mean_squared_error
RMSE = mean_squared_error(y_test, result)**0.5
print("Rmse=",RMSE)

Mean Absolute % error =  12.3967424977
Rmse= 8085.85120835


In [77]:
# Examine available columns
#x=df[['DEP_INC_PCT_LO','PREDDEG','PCIP12','NPT41_PUB','NPT42_PUB','UGDS_ASIAN','FIRSTGEN_WDRAW_ORIG_YR3_RT']]
#y=df[['md_earn_wne_p6']]

In [105]:
################## ALGORITHMS ##############################
########### ############ ############ ############# ########
####### ####### ######### ############### ########### #######

In [119]:
##################### 1) LINEAR REGRESSION ################ 
###########################################################
## Apply KPCA
## Kernel PCA
from sklearn.decomposition import PCA, KernelPCA
kpca = KernelPCA(kernel="rbf",gamma=10,n_components=400)
fit=kpca.fit(x_train)
x_train1 = kpca.transform(x_train)
x_test1 = kpca.transform(x_test)

# Linear Regression

def augment(x):
    numRows = x.shape[0]
    onesVec = np.ones((numRows, 1))
    y = np.append(onesVec, x, 1)
    return y

def normalize(X, mean_X, std_X):
    '''
        Normalize our sample set wrt to mean and standard deviation
    '''
    num_samples = X.shape[0]
    for i in range(num_samples):
        X[i,:] = X[i, :] - mean_X
        X[i,:] = X[i, :]/std_X
    return X
        


def lr_train(x_train, y_train, mean_X, std_X, alpha=0.005, epsilon = 0.002, num_iterations=500):
    '''
        Finding theta parameters for linear regression
    '''
    X = copy.deepcopy(x_train)
    Y = copy.deepcopy(y_train)
    X = normalize(X, mean_X, std_X)
    X = augment(X)
#     print('Dim of X', X.shape)
    theta = np.ones(X.shape[1])
    num_samples = X.shape[0]
#     for k in range(num_iterations):
    counter = 0
    while True:
        idx = random.randint(0, num_samples - 1)
#         print('Dim of theta', theta.shape)
#         print('Dim of X[idx]', X[idx].shape)
        h = theta.dot(X[idx])
        diff = Y[idx] - h

        for j in range(theta.shape[0]):
            theta[j] = theta[j] + alpha * diff * X[idx][j]
#             print('thetaj:',j, ' -> ',theta[j])
        error_value = alpha*diff
        if error_value < 0:
            error_value *= -1
        if error_value < epsilon or counter > num_iterations:
            break
        counter += 1
#         print('Counter: ', counter)
    return theta

def lr_test(x_test, theta, mean_X, std_X):
    X = copy.deepcopy(x_test)
    X = normalize(X, mean_X, std_X)
#     print('x_normalize', X)
    X = augment(X)
#     print('x_augment', X)
    num_samples = X.shape[0]
    output = np.ones((num_samples, 1))
    for i in range(num_samples):
        idx = i
#         print('theta', theta)
#         print('X', X[idx])
        output[idx] = theta.dot(X[idx])
    return output

import copy

mean_X = np.mean(x_train1, 0)
std_X = np.std(x_train1,0)

theta = lr_train(x_train1, y_train, mean_X, std_X)
# print('Theta', theta)
# print('x_test', x_test)
result = lr_test(x_test1, theta, mean_X, std_X)

percent_error=mean_absolute_percentage_error(y_test,result)
print("Mean Absolute % error = ",percent_error)

## RMSE
from sklearn.metrics import mean_squared_error
RMSE = mean_squared_error(y_test, result)**0.5
print("Rmse=",RMSE)


Mean Absolute % error =  15.734283616
Rmse= 15717.5884852


In [18]:
x_train=np.array(x_train)
x_test=np.array(x_test)
np.max(y_train)

133600

In [124]:
############  2) WEIGHTED LINEAR REGRESSION ##############
#########################################################
def lwe_weight_matrix(X, point, c):
    n = X.shape[0]
    weight = np.eye(n)
    for i in range(n):
        diff = X[i] - point
        diff_mod = LA.norm(diff)**2
#         if i == 1:
#             print('dif mod', diff_mod)
#             print('Xi',X[i])
#             print('point',point)
        value = np.exp(-1*diff_mod*0.5/c**2)
        weight[i, i] = value
    return weight

def lwr_test_point(x_train, y_train, point, c):
#     X = copy.deepcopy(x_train)
#     Y = copy.deepcopy(y_train)
    X = x_train
    Y = y_train
    weight = lwe_weight_matrix(X, point,c)
#     print(weight)
    t1 = X.T.dot(weight)
    t1 = t1.dot(X)
    t1 = LA.inv(t1)
    t2 = X.T.dot(weight)
    t2 = t2.dot(Y)
    beta = t1.dot(t2)
    return point.dot(beta)

def lwr_test(x_train, y_train, x_test, c=5):
    print('Shapes: Xtrain ', x_train.shape, 'Shapes: Xtest', x_test.shape)
    result = np.ones(x_test.shape[0])
    X = copy.deepcopy(x_train)
    Y = copy.deepcopy(y_train)
    X_Test = copy.deepcopy(x_test)
    
    mean_X = np.mean(X,0)
    std_X = np.std(X,0)
    
    X = normalize(X, mean_X, std_X)
    X_Test = normalize(X_Test, mean_X, std_X)
   
    print('result shape:', result.shape)
    for i in range(x_test.shape[0]):
        result[i] = lwr_test_point(X, Y, X_Test[i], c)
        print(i,' : ', result[i])
#         print('For ', X_Test[i])
    return result
    

# testing locally weighted linear regression
ktil = 10
result = lwr_test(x_train,y_train,x_test[0:ktil,:]) # tests point by point and not all at once

print('ytest shape, result', y_test[0:ktil].shape, result.shape)
percent_error=mean_absolute_percentage_error(y_test[0:ktil],result)
print("Mean Absolute % error = ",percent_error)

print('Y test', y_test[0:ktil])
print('Predicting Y', result)
## RMSE
from sklearn.metrics import mean_squared_error
RMSE = mean_squared_error(y_test[0:ktil], result)**0.5
print("Rmse=",RMSE)


Shapes: Xtrain  (6000, 458) Shapes: Xtest (10, 458)
result shape: (10,)
0  :  21065.6965118
1  :  18365.2812411
2  :  21027.7131905
3  :  27822.0592836
4  :  18366.6571766
5  :  21643.3143831
6  :  17452.4692454
7  :  26353.9469761
8  :  38196.7184984
9  :  38194.743035
ytest shape, result (10, 1) (10,)
Mean Absolute % error =  31.2729575358
Y test [[20500]
 [19500]
 [19400]
 [25500]
 [16800]
 [23300]
 [29000]
 [24800]
 [38200]
 [38200]]
Predicting Y [ 21065.69651181  18365.28124112  21027.71319049  27822.05928363
  18366.65717655  21643.31438311  17452.46924544  26353.94697609
  38196.71849843  38194.74303496]
Rmse= 3880.82835011


In [18]:
############## 3)  KNN Regression ################
##################################################

K=10
train=df[:6000]
test=df[6000:]
test=test.as_matrix()
train=train.as_matrix()

result_knn=[]
for i in range(0,len(test)): ## Take each test point and calculate ditance to each train point
    D=[] ## The distance matrix
    for j in range(0,len(train)): ## Distance to each train point
        dist = np.linalg.norm(test[i]-train[j])
        D.append(dist)
    D=np.array(D)
    indexes = D.argsort()[::1]
    top_k = y[ indexes[0:K]] ## Will contain K nearest neighbours
    
    # Calculate the average of the prediction variable
    price=0
    for j in range(0,K):
        price+=top_k.iloc[j]
    price=price/K
    result_knn.append(price)  

## Mean Absolute Percentage Error
from numpy import *
def mean_absolute_percentage_error(y_true, y_pred):
    a=np.abs(y_true - y_pred)
    where_are_NaNs = isnan(a)
    a[where_are_NaNs] = 0
#     a[a == inf] = 0
    b=( a/ y_true)
    b[b == inf] = 0
    where_are_NaNs = isnan(b)
    b[where_are_NaNs] = 0
    b=np.mean(b)
    return b * 100

percent_error=mean_absolute_percentage_error(y_test,np.array(result_knn))
print("Mean Absolute % error = ",percent_error)

## RMSE
from sklearn.metrics import mean_squared_error
RMSE = mean_squared_error(y_test, result_knn)**0.5
print("Rmse=",RMSE)

Mean Absolute % error =  23.042434652
Rmse= 13478.5492864


Dim of weights (6000, 6000)
Dim of y (1, 6000)
[[ -4.50380725e-14]]


In [15]:
################## 4) Neural Networks ###################
#########################################################
# Neural Nets

def sigmoid(x):
    k = 1
    ex = math.e ** x
    val = k*ex/(1 + ex)
    return val

def dif_sigmoid(x):
    k = 1
    sig = sigmoid(x)
    val = k*sig * (1 - sig)
    return val

def relu(x):
    return np.log(1+np.exp(x))

def dif_relu(x):
    return 1/(1+np.exp(-1.0*x))

def activation(x):
    return sigmoid(x)

def dif_activation(x):
    return dif_sigmoid(x)


def feed_forward(x, wh, wo, nh, no):
    y = []
    z = []
    neth = []
    neto = []
    aug_x = np.append(1, x)
    for j in range(nh):
        net_j = aug_x.dot(wh[j, :])
        neth.append(net_j)
#         y_j = sigmoid(net_j)
        y_j = activation(net_j)
        y.append(y_j)

    yk = [1]
    yk = yk + y
    # y.insert(0, 1)
    aug_y = np.array(yk)

    for k in range(no):
        # print 'aug_y: ', aug_y
        # print 'wok: ', wo[k, :]
        net_k = aug_y.dot(wo[k, :])
        neto.append(net_k)
        # print 'net_k', net_k
#         z_k = sigmoid(net_k)
        z_k = activation(net_k)
        z.append(z_k)

    z = np.array(z)
    return z, y, neth, neto


def stochastic_backprop(samples, outputs, d, nh, no, theta, eta):
    random.seed(4)
    wh = 2 * np.random.rand(nh, d+1) - 1
    wo = 2 * np.random.rand(no, nh+1) - 1
    num_samples = samples.shape[0]
    # print 'Num samples: ', num_samples
    count = 0
    delo = []
    delk = []

    while True:
        delo = []
        idx = random.randint(0, num_samples - 1)
        # digit = digits[idx]
        # t = target_value(digit)
        t = outputs[idx]
        x = np.array(samples[idx])
        z, y, neth, neto = feed_forward(x, wh, wo, nh, no)
        aug_x = np.append(1, x)

        yk = [1]
        yk = yk + y
        # y.insert(0, 1)
        aug_y = np.array(yk)

        # x = aug_x
        for k in range(no):
            dif = t[k] - z[k]
#             fdash = dif_sigmoid(neto[k])
            fdash = dif_activation(neto[k])
            dell = dif * fdash
            delo.append(dell)
            for j in range(nh+1):
                delta = eta * dell * aug_y[j]
                wo[k, j] = wo[k, j] + delta

        for j in range(nh):
#             fdash = dif_sigmoid(neth[j])
            fdash = dif_activation(neth[j])
            sigma = 0
            for k in range(no):
                sigma += (wo[k, j] * delo[k])
            dell = fdash * sigma
            for i in range(d+1):                                                                         
                delta = eta * dell * aug_x[i]
                wh[j, i] = wh[j, i] + delta

        z, y, neth, neto = feed_forward(x, wh, wo, nh, no)
        diff = t - z
#         print('dif', diff)
        mod = np.linalg.norm(diff)
        j = 0.5 * mod * mod
        if count > 10000:
            print(j)
            break
        # if count > 5000:
        # 	break
        count += 1

    print('Count: ', count)

    return wo,wh

######

#####

def approx(x):
    if x< 0 or x >1:
        print('ERROR: Value approximated greater than 1: ', x)
    if x < 0.5:
        return 0
    else:
        return 1


def nn_train(X, Y):
    d = X.shape[1]
#     print('DDl', d)
    nh = 8
    no = Y.shape[1]
    theta = 0.000001
    eta = 0.125
    wo, wh = stochastic_backprop(X, Y, d, nh, no, theta, eta)
    return wo, wh, nh, no
    
def nn_test(point, wo, wh, nh, no):
    z, y, neth, neto = feed_forward(point, wh, wo, nh, no)
    return z


wo, wh, nh, no = nn_train(x_train, y_train)

for i in range(50):
    result = nn_test(x_test[i], wo, wh, nh, no)
    print('Original->', y_test[i], ' Predicted-> ', result)


nan
Count:  10001
Original-> [20500]  Predicted->  [ nan]
Original-> [19500]  Predicted->  [ nan]
Original-> [19400]  Predicted->  [ nan]
Original-> [25500]  Predicted->  [ nan]
Original-> [16800]  Predicted->  [ nan]
Original-> [23300]  Predicted->  [ nan]
Original-> [29000]  Predicted->  [ nan]
Original-> [24800]  Predicted->  [ nan]
Original-> [38200]  Predicted->  [ nan]
Original-> [38200]  Predicted->  [ nan]
Original-> [19200]  Predicted->  [ nan]
Original-> [19400]  Predicted->  [ nan]
Original-> [17700]  Predicted->  [ nan]
Original-> [25300]  Predicted->  [ nan]
Original-> [25700]  Predicted->  [ nan]
Original-> [21200]  Predicted->  [ nan]
Original-> [18600]  Predicted->  [ nan]
Original-> [38200]  Predicted->  [ nan]
Original-> [34100]  Predicted->  [ nan]
Original-> [27800]  Predicted->  [ nan]
Original-> [20700]  Predicted->  [ nan]
Original-> [36100]  Predicted->  [ nan]
Original-> [23200]  Predicted->  [ nan]
Original-> [21100]  Predicted->  [ nan]
Original-> [21100]  Pr

In [35]:
################## 5) SVM Regression ###################
########################################################

from sklearn.svm import SVR
clf = SVR(C=1.0, epsilon=0.2,kernel='linear')
# SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma='auto',
#     kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
clf.fit(x_train,y_train.tolist())
print('Fitted')
result=clf.predict(x_test)
print('Predicted')

## Mean Absolute Percentage Error
from numpy import inf
def mean_absolute_percentage_error(y_true, y_pred):
    a=np.abs(y_true - y_pred)
    a[a == inf] = 0
    b=( a/ y_true)
    b[b == inf] = 0
    b=np.mean(b)
    return b * 100

percent_error=mean_absolute_percentage_error(y_test,result)
print("Mean Absolute % error = ",percent_error)


## RMSE
from sklearn.metrics import mean_squared_error
RMSE = mean_squared_error(y_test, result)**0.5
print("Rmse=",RMSE)

/home/prakhar/Downloads/enter/lib/python3.5/site-packages/sklearn/svm/base.py:216: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True).astype(np.float64)


Fitted
Predicted
Mean Absolute % error =  17.068129199
Rmse= 16783.402844


In [211]:
##########               ########
########## PLOtting Code ########
##########              ########

#########3 Correlation List
# copydf=df.copy(deep=True)
# copydf.loc[:,copydf.shape[1]]=y 
# corr = copydf.select_dtypes(include = ['float64', 'int64']).iloc[:, 1:].corr()
# a=[]
# cor_dict = corr[copydf.shape[1]-1].to_dict()
# print("List the numerical features decendingly by their correlation with Earnings:\n")
# for ele in sorted(cor_dict.items(), key = lambda x: -abs(x[1])):
# #     print("{0}:\t\t\t\t\t\t\t{1}".format(*ele))
#         a.append([ele[0],ele[1]])
# b=pd.DataFrame(a)
# b

########## Graph and General info of Prediction Variables
# print("Some Statistics of the Income:\n")
# print(y2.describe())
# sns.distplot(y2, kde = False, color = 'b', hist_kws={'alpha': 0.9})


########## HEAT MAP #############
# copydf=df.as_matrix()
# copydf=copydf[:,:20]
# copydf=pd.DataFrame(copydf)
# copydf.loc[:,copydf.shape[1]]=y
# corr = copydf.select_dtypes(include = ['float64', 'int64']).iloc[:, 1:].corr()
# plt.figure(figsize=(12, 12))
# sns.heatmap(corr, vmax=1, square=True)
### 20 is our median earning

########## Correlation
# sns.regplot(x=df['PCIP45'], y=y, data = df, color = 'Orange')

########## Asian and Predominant degree awarded
# fig = plt.figure(2, figsize=(9, 7))
# plt.subplot(211)
# plt.scatter(df.UGDS_ASIAN.values,y)
# plt.title('Asian Undergrads')
# plt.subplot(212)
# plt.scatter(df.PREDDEG.values, y)
# plt.title('Predominant Degree')
# fig.text(-0.01, 0.5, y, va = 'center', rotation = 'vertical', fontsize = 12)
# plt.tight_layout()


############# OLD HEAT MAP
# # Heat Map
# sns.set(style="white")
# # df1=df[['DEP_INC_PCT_LO','PREDDEG','PCIP12','NPT41_PUB','NPT42_PUB','UGDS_ASIAN','FIRSTGEN_WDRAW_ORIG_YR3_RT','md_earn_wne_p6']]
# # Compute the correlation matrix
# corr = df1.corr()
# # print(corr)
# # Generate a mask for the upper triangle
# mask = np.zeros_like(corr, dtype=np.bool)
# mask[np.triu_indices_from(mask)] = True
# # Set up the matplotlib figure
# f, ax = plt.subplots(figsize=(11, 9))
# # Generate a custom diverging colormap
# cmap = sns.diverging_palette(220, 10, as_cmap=True)
# # Draw the heatmap with the mask and correct aspect ratio
# sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3,
#             square=True, xticklabels=5, yticklabels=5,
#             linewidths=.5, cbar_kws={"shrink": .5}, ax=ax)

